In [1]:
# analytics
import pandas as pd 
import numpy as np
import scipy.stats as stats
import statsmodels.formula.api as smf
#spatial 
import osmnx as ox
import geopandas as gpd
import contextily as cx
# plotting 
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
#settings
import warnings

In [2]:
# import data
path = '/Users/philip/Documents/ESE/ESE_thesis/flood_experience/data/export/clean_n.csv'
df_n = pd.read_csv(path)
df_n.columns

Index(['id', 'state', 'zipcode', 'geographic_division', 'census_region',
       'county', 'experience', 'supplies', 'insured', 'involved',
       'learned_routes', 'made_plan', 'made_safer', 'planned_neighbors',
       'practiced_drills', 'documents', 'rainy_day', 'alerts',
       'family_communication', 'none', 'dont_know', 'age', 'sex', 'education',
       'race', 'homeownership', 'income', 'rentmortgage', 'rurality',
       'hazard_weight', 'geometry', 'zip_count'],
      dtype='object')

In [3]:
path = '/Users/philip/Documents/ESE/ESE_thesis/flood_experience/data/export/clean_k.csv'
df_k = pd.read_csv(path)
df_k.columns

Index(['id', 'state', 'zipcode', 'geographic_division', 'census_region',
       'county', 'awareness', 'perception', 'experience', 'floodzone',
       'efficacy', 'supplies', 'insured', 'involved', 'learned_routes',
       'made_plan', 'made_safer', 'planned_neighbors', 'practiced_drills',
       'documents', 'rainy_day', 'alerts', 'family_communication', 'none',
       'dont_know', 'age', 'sex', 'education', 'race', 'homeownership',
       'income', 'rentmortgage', 'rurality', 'hazard_weight', 'geometry',
       'zip_count'],
      dtype='object')

#### We will go through these outcome variables one by one

structural adaptation: 
- made_safer

non-structural adaptation: 
- insurance
- learned_routes
- supplies
- involved
- made_plan
- practiced_drills
- alerts
- family_communication

In [4]:
def probit(functions, data):
    results_list = []
    for func in functions:
        model = smf.probit(formula=func, data=data).fit(disp=0)

        df_model = pd.DataFrame({
            'effect': model.params,               
            'p': model.pvalues,                   
            'pseudoR_2': model.prsquared,
            'LLPr': model.llr_pvalue,
            'BIC': model.bic  
        })
        df_model.index = pd.MultiIndex.from_product([[func], df_model.index], names=['function', 'beta'])
        results_list.append(df_model)
    results = pd.concat(results_list)
    return results


In [5]:
#duplicate but with logit
def logit(functions, data):
    results_list = []
    for func in functions:
        model = smf.logit(formula=func, data=data).fit(disp=0)
        marg_effects = model.get_margeff().summary_frame()

        df_model = pd.DataFrame({
            'effect': model.params,               
            'p': model.pvalues,                   
            'marginal_effect': marg_effects['dy/dx'],
            'pseudoR_2': model.prsquared,
            'LLPr': model.llr_pvalue,
            'BIC': model.bic  
        })
        df_model.index = pd.MultiIndex.from_product([[func], df_model.index], names=['function', 'beta'])
        results_list.append(df_model)
    results = pd.concat(results_list)
    return results

In [6]:

functions = [
    'made_safer ~ age + income + education + homeownership',
    'insured ~ age + income + education + homeownership',
    'learned_routes ~ age + income + education + homeownership',
    'supplies ~ age + income + education + homeownership',
    'involved ~ age + income + education + homeownership',
    'made_plan ~ age + income + education + homeownership',
    'practiced_drills ~ age + income + education + homeownership',
    'alerts ~ age + income + education + homeownership',
    'family_communication ~ age + income + education + homeownership'    
]

In [7]:
probit(functions=functions, data=df_n)

effect  \
function                                           beta                          
made_safer ~ age + income + education + homeown... Intercept     -5.956566e-01   
                                                   age           -4.089442e-03   
                                                   income        -2.885281e-06   
                                                   education     -3.412994e-02   
                                                   homeownership  3.207616e-01   
insured ~ age + income + education + homeownership Intercept     -6.864550e-01   
                                                   age           -8.500531e-03   
                                                   income        -1.125904e-06   
                                                   education      6.156973e-02   
                                                   homeownership  4.125036e-01   
learned_routes ~ age + income + education + hom... Intercept     -7.198322e-01   
                                                   age            1.771067e-04   
                                                   income         4.071082e-07   
                                                   education     -8.728690e-03   
                                                   homeownership -7.999015e-02   
supplies ~ age + income + education + homeowner... Intercept     -1.607653e-01   
                                                   age           -1.048431e-02   
                                                   income        -1.633306e-06   
                                                   education     -3.769223e-02   
                                                   homeownership  2.247312e-01   
involved ~ age + income + education + homeowner... Intercept     -6.086076e-01   
                                                   age           -1.857688e-02   
                                                   income         1.264398e-06   
                                                   education      8.896976e-02   
                                                   homeownership -3.912580e-01   
made_plan ~ age + income + education + homeowne... Intercept     -4.005450e-01   
                                                   age           -6.141770e-03   
                                                   income         6.552215e-07   
                                                   education     -4.060618e-02   
                                                   homeownership -5.220720e-03   
practiced_drills ~ age + income + education + h... Intercept     -5.977069e-01   
                                                   age           -1.853035e-02   
                                                   income        -1.503103e-06   
                                                   education      4.511960e-02   
                                                   homeownership  9.948108e-02   
alerts ~ age + income + education + homeownership  Intercept     -1.030586e+00   
                                                   age            2.011578e-03   
                                                   income        -7.945906e-07   
                                                   education      1.852343e-02   
                                                   homeownership  8.197137e-02   
family_communication ~ age + income + education... Intercept     -9.337578e-01   
                                                   age           -1.062838e-02   
                                                   income        -5.488783e-07   
                                                   education      8.046730e-02   
                                                   homeownership  1.776218e-02   

                                                                         p  \
function                                           beta                      
made_safer ~ age + income + education + homeown... Intercept 

In [8]:
logit(functions=functions, data=df_n)

effect  \
function                                           beta                          
made_safer ~ age + income + education + homeown... Intercept     -9.408183e-01   
                                                   age           -7.476948e-03   
                                                   education     -6.067294e-02   
                                                   homeownership  5.711845e-01   
                                                   income        -5.201298e-06   
insured ~ age + income + education + homeownership Intercept     -1.115998e+00   
                                                   age           -1.522186e-02   
                                                   education      1.087397e-01   
                                                   homeownership  7.275066e-01   
                                                   income        -1.957167e-06   
learned_routes ~ age + income + education + hom... Intercept     -1.176452e+00   
                                                   age            3.322448e-04   
                                                   education     -1.501308e-02   
                                                   homeownership -1.371368e-01   
                                                   income         6.849536e-07   
supplies ~ age + income + education + homeowner... Intercept     -2.105574e-01   
                                                   age           -1.812408e-02   
                                                   education     -6.564697e-02   
                                                   homeownership  3.831917e-01   
                                                   income        -2.864610e-06   
involved ~ age + income + education + homeowner... Intercept     -9.044477e-01   
                                                   age           -3.672741e-02   
                                                   education      1.758586e-01   
                                                   homeownership -7.424869e-01   
                                                   income         2.300738e-06   
made_plan ~ age + income + education + homeowne... Intercept     -6.241621e-01   
                                                   age           -1.054272e-02   
                                                   education     -7.013162e-02   
                                                   homeownership -6.601460e-03   
                                                   income         1.025637e-06   
practiced_drills ~ age + income + education + h... Intercept     -9.130370e-01   
                                                   age           -3.709485e-02   
                                                   education      1.065308e-01   
                                                   homeownership  1.968828e-01   
                                                   income        -2.754673e-06   
alerts ~ age + income + education + homeownership  Intercept     -1.721770e+00   
                                                   age            3.481480e-03   
                                                   education      3.357778e-02   
                                                   homeownership  1.448055e-01   
                                                   income        -1.363827e-06   
family_communication ~ age + income + education... Intercept     -1.530877e+00   
                                                   age           -2.101251e-02   
                                                   education      1.579868e-01   
                                                   homeownership  5.766728e-02   
                                                   income        -1.115632e-06   

                                                                         p  \
function                                           beta                      
made_safer ~ age + income + education + homeown... Intercept 